In [3]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load a YOLO model
model = YOLO("yolo11s.pt")

# Open the input video
input_video_path = "5.mp4"
output_video_path = "output_video3.mp4"
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Keep the original FPS but update every 60 frames
frame_counter = 0
update_interval = 2

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Define a threshold for traffic density
traffic_density_threshold = 4  # Adjust this value based on your needs

# Initialize variables for polygon drawing
polygon_points = []
drawing_polygon = False
vehicle_count = 0  # Store vehicle count for every update interval

# Mouse callback function to draw polygon
def draw_polygon(event, x, y, flags, param):
    global drawing_polygon, polygon_points, frame

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing_polygon = True
        polygon_points.append((x, y))
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

    elif event == cv2.EVENT_MOUSEMOVE and drawing_polygon:
        if len(polygon_points) > 1:
            temp_frame = frame.copy()
            cv2.line(temp_frame, polygon_points[-1], (x, y), (0, 255, 0), 2)
            cv2.imshow('Draw Polygon', temp_frame)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing_polygon = False
        polygon_points.append((x, y))
        if len(polygon_points) > 1:
            cv2.line(frame, polygon_points[-2], polygon_points[-1], (0, 255, 0), 2)

# Read the first frame to draw the polygon
ret, frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    exit()

# Create a window and bind the mouse callback function
cv2.namedWindow('Draw Polygon')
cv2.setMouseCallback('Draw Polygon', draw_polygon)

# Draw the polygon on the first frame
while True:
    cv2.imshow('Draw Polygon', frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # Press 'ESC' to exit
        break
    elif k == 13:  # Press 'Enter' to confirm the polygon
        break

cv2.destroyAllWindows()

# Check if the polygon was drawn
if len(polygon_points) < 3:
    print("Error: Polygon must have at least 3 points.")
    cap.release()
    out.release()
    exit()

# Convert the list of points to a NumPy array and ensure they are integers
polygon_points = np.array(polygon_points, dtype=np.int32)
polygon_points = polygon_points.reshape((-1, 1, 2))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    
    # Only update vehicle count every 60 frames
    if frame_counter % update_interval == 0:
        # Convert frame to RGB for YOLO
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform prediction
        results = model.predict(frame_rgb, imgsz=1504, classes=[1, 2, 3, 5, 7])

        vehicle_count = 0  # Reset vehicle count for the update interval

        # Count vehicles inside the polygon
        if isinstance(results, list):
            for result in results:
                for det in result.boxes:
                    x1, y1, x2, y2 = map(int, det.xyxy[0].tolist())
                    
                    # Check if the center of the detected object is inside the polygon
                    box_center = (int((x1 + x2) // 2), int((y1 + y2) // 2))
                    if cv2.pointPolygonTest(polygon_points, box_center, False) >= 0:
                        vehicle_count += 1

    # Fill the polygon with semi-transparent color based on vehicle count
    if vehicle_count > traffic_density_threshold:
        color = (0, 255, 0)  # Green for high traffic
        message = "Signal Turn ON"
    else:
        color = (0, 0, 255)  # Red for low traffic
        message = "Signal Turn OFF"

    # Create a filled polygon with transparency
    overlay = frame.copy()
    cv2.fillPoly(overlay, [polygon_points], color)
    
    # Blend the overlay with the original frame
    alpha = 0.4  # Transparency factor
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    # Draw polygon outline
    cv2.polylines(frame, [polygon_points], isClosed=True, color=(255, 255, 255), thickness=2)

    # Display the vehicle count at the top of the frame
    cv2.putText(frame, f'Vehicles Detected: {vehicle_count}', (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3, cv2.LINE_AA)
    
    # Display the signal status in an attractive manner
    cv2.putText(frame, message, (frame_width // 2 - 150, frame_height // 10),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 5, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()


0: 864x1504 18 cars, 6 trucks, 1124.7ms
Speed: 27.1ms preprocess, 1124.7ms inference, 23.3ms postprocess per image at shape (1, 3, 864, 1504)

0: 864x1504 16 cars, 4 trucks, 816.5ms
Speed: 19.0ms preprocess, 816.5ms inference, 2.0ms postprocess per image at shape (1, 3, 864, 1504)

0: 864x1504 17 cars, 4 trucks, 740.6ms
Speed: 20.9ms preprocess, 740.6ms inference, 3.0ms postprocess per image at shape (1, 3, 864, 1504)

0: 864x1504 15 cars, 5 trucks, 708.6ms
Speed: 23.9ms preprocess, 708.6ms inference, 3.0ms postprocess per image at shape (1, 3, 864, 1504)

0: 864x1504 17 cars, 5 trucks, 743.5ms
Speed: 20.3ms preprocess, 743.5ms inference, 3.0ms postprocess per image at shape (1, 3, 864, 1504)

0: 864x1504 16 cars, 1 motorcycle, 5 trucks, 861.3ms
Speed: 20.9ms preprocess, 861.3ms inference, 3.0ms postprocess per image at shape (1, 3, 864, 1504)

0: 864x1504 14 cars, 1 motorcycle, 4 trucks, 771.1ms
Speed: 19.9ms preprocess, 771.1ms inference, 4.0ms postprocess per image at shape (1, 3, 

### Keeping the signal turn on for specific time

In [2]:
import cv2
from ultralytics import YOLO
import numpy as np
import time

# Load a YOLO model
model = YOLO("yolo11s.pt")

# Open the input video
input_video_path = "5.mp4"
output_video_path = "output_video2.mp4"
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Keep the original FPS but update every 60 frames
frame_counter = 0
update_interval = 20

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Define a threshold for traffic density
traffic_density_threshold = 4  # Adjust this value based on your needs

# Initialize variables for polygon drawing
polygon_points = []
drawing_polygon = False
vehicle_count = 0  # Store vehicle count for every update interval

# Variable to store the signal duration in seconds
signal_duration = 15  # Set the desired duration in seconds

# Variable to track the start time of the signal
signal_start_time = None

# Variable to track the signal state
signal_state = "OFF"

# Mouse callback function to draw polygon
def draw_polygon(event, x, y, flags, param):
    global drawing_polygon, polygon_points, frame

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing_polygon = True
        polygon_points.append((x, y))
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

    elif event == cv2.EVENT_MOUSEMOVE and drawing_polygon:
        if len(polygon_points) > 1:
            temp_frame = frame.copy()
            cv2.line(temp_frame, polygon_points[-1], (x, y), (0, 255, 0), 2)
            cv2.imshow('Draw Polygon', temp_frame)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing_polygon = False
        polygon_points.append((x, y))
        if len(polygon_points) > 1:
            cv2.line(frame, polygon_points[-2], polygon_points[-1], (0, 255, 0), 2)

# Read the first frame to draw the polygon
ret, frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    exit()

# Create a window and bind the mouse callback function
cv2.namedWindow('Draw Polygon')
cv2.setMouseCallback('Draw Polygon', draw_polygon)

# Draw the polygon on the first frame
while True:
    cv2.imshow('Draw Polygon', frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # Press 'ESC' to exit
        break
    elif k == 13:  # Press 'Enter' to confirm the polygon
        break

cv2.destroyAllWindows()

# Check if the polygon was drawn
if len(polygon_points) < 3:
    print("Error: Polygon must have at least 3 points.")
    cap.release()
    out.release()
    exit()

# Convert the list of points to a NumPy array and ensure they are integers
polygon_points = np.array(polygon_points, dtype=np.int32)
polygon_points = polygon_points.reshape((-1, 1, 2))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    
    # Only update vehicle count every 60 frames
    if frame_counter % update_interval == 0:
        # Convert frame to RGB for YOLO
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform prediction
        results = model.predict(frame_rgb, imgsz=640, classes=[1, 2, 3, 5, 7])

        vehicle_count = 0  # Reset vehicle count for the update interval

        # Count vehicles inside the polygon
        if isinstance(results, list):
            for result in results:
                for det in result.boxes:
                    x1, y1, x2, y2 = map(int, det.xyxy[0].tolist())
                    
                    # Check if the center of the detected object is inside the polygon
                    box_center = (int((x1 + x2) // 2), int((y1 + y2) // 2))
                    if cv2.pointPolygonTest(polygon_points, box_center, False) >= 0:
                        vehicle_count += 1

    # Check if the signal should be turned ON based on vehicle count
    if vehicle_count > traffic_density_threshold and signal_state == "OFF":
        signal_start_time = time.time()  # Record the start time
        signal_state = "ON"

    # Check if the signal duration has passed
    if signal_start_time is not None and time.time() - signal_start_time >= signal_duration:
        signal_state = "OFF"
        signal_start_time = None  # Reset the start time

    # Determine the color and message based on the signal state
    if signal_state == "ON":
        color = (0, 255, 0)  # Green for high traffic
        message = "Signal Turn ON"
    else:
        color = (0, 0, 255)  # Red for low traffic
        message = "Signal Turn OFF"

    # Create a filled polygon with transparency
    overlay = frame.copy()
    cv2.fillPoly(overlay, [polygon_points], color)
    
    # Blend the overlay with the original frame
    alpha = 0.4  # Transparency factor
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    # Draw polygon outline
    cv2.polylines(frame, [polygon_points], isClosed=True, color=(255, 255, 255), thickness=2)

    # Display the vehicle count at the top of the frame
    # cv2.putText(frame, f'Vehicles Count: {vehicle_count}', (50, 50), 
    #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3, cv2.LINE_AA)
    
    # Display the signal status in an attractive manner
    cv2.putText(frame, message, (frame_width // 2 - 150, frame_height // 5),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 5, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 9 cars, 3 trucks, 189.5ms
Speed: 5.0ms preprocess, 189.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 3 trucks, 183.5ms
Speed: 2.0ms preprocess, 183.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 190.5ms
Speed: 4.0ms preprocess, 190.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 226.4ms
Speed: 4.0ms preprocess, 226.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 184.5ms
Speed: 4.0ms preprocess, 184.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 163.6ms
Speed: 3.0ms preprocess, 163.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 trucks, 179.5ms
Speed: 3.0ms preprocess, 179.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 3 trucks, 150.6ms
Speed: 3.0m

In [18]:
import cv2
from ultralytics import YOLO
import numpy as np
import time

# Load a YOLO model
model = YOLO("yolo11s.pt")

# Open the input video
input_video_path = "6.mp4"
output_video_path = "output_video6.mp4"
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Keep the original FPS but update every 60 frames
frame_counter = 0
update_interval = 30

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Define a threshold for traffic density
traffic_density_threshold = 9  # Adjust this value based on your needs

# Initialize variables for polygon drawing
polygon_points = []
drawing_polygon = False
vehicle_count = 0  # Store vehicle count for every update interval

# Define durations for signal ON and OFF states
signal_on_duration = 18  # Duration in seconds for signal to stay ON
signal_off_duration = 7  # Duration in seconds for signal to stay OFF

# Variables to track the start time of the signal
signal_start_time = None
signal_state = "OFF"

# Mouse callback function to draw polygon
def draw_polygon(event, x, y, flags, param):
    global drawing_polygon, polygon_points, frame

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing_polygon = True
        polygon_points.append((x, y))
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

    elif event == cv2.EVENT_MOUSEMOVE and drawing_polygon:
        if len(polygon_points) > 1:
            temp_frame = frame.copy()
            cv2.line(temp_frame, polygon_points[-1], (x, y), (0, 255, 0), 2)
            cv2.imshow('Draw Polygon', temp_frame)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing_polygon = False
        polygon_points.append((x, y))
        if len(polygon_points) > 1:
            cv2.line(frame, polygon_points[-2], polygon_points[-1], (0, 255, 0), 2)

# Read the first frame to draw the polygon
ret, frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    exit()

# Create a window and bind the mouse callback function
cv2.namedWindow('Draw Polygon')
cv2.setMouseCallback('Draw Polygon', draw_polygon)

# Draw the polygon on the first frame
while True:
    cv2.imshow('Draw Polygon', frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # Press 'ESC' to exit
        break
    elif k == 13:  # Press 'Enter' to confirm the polygon
        break

cv2.destroyAllWindows()

# Check if the polygon was drawn
if len(polygon_points) < 3:
    print("Error: Polygon must have at least 3 points.")
    cap.release()
    out.release()
    exit()

# Convert the list of points to a NumPy array and ensure they are integers
polygon_points = np.array(polygon_points, dtype=np.int32)
polygon_points = polygon_points.reshape((-1, 1, 2))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    
    # Only update vehicle count every 60 frames
    if frame_counter % update_interval == 0:
        # Convert frame to RGB for YOLO
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform prediction
        results = model.predict(frame_rgb, imgsz=1704, classes=[1, 2, 3, 5, 7])

        vehicle_count = 0  # Reset vehicle count for the update interval

        # Count vehicles inside the polygon
        if isinstance(results, list):
            for result in results:
                for det in result.boxes:
                    x1, y1, x2, y2 = map(int, det.xyxy[0].tolist())
                    
                    # Check if the center of the detected object is inside the polygon
                    box_center = (int((x1 + x2) // 2), int((y1 + y2) // 2))
                    if cv2.pointPolygonTest(polygon_points, box_center, False) >= 0:
                        vehicle_count += 1

    # Check if the signal should be turned ON based on vehicle count
    if vehicle_count > traffic_density_threshold and signal_state == "OFF":
        signal_start_time = time.time()  # Record the start time
        signal_state = "ON"

    # Check if the signal duration has passed
    if signal_state == "ON" and signal_start_time is not None and time.time() - signal_start_time >= signal_on_duration:
        signal_state = "OFF"
        signal_start_time = time.time()  # Reset the start time for OFF state

    # Check if the signal OFF duration has passed
    if signal_state == "OFF" and signal_start_time is not None and time.time() - signal_start_time >= signal_off_duration:
        signal_state = "ON"
        signal_start_time = time.time()  # Reset the start time for ON state

    # Determine the color and message based on the signal state
    if signal_state == "ON":
        color = (0, 255, 0)  # Green for high traffic
        message = "Signal Turn ON"
    else:
        color = (0, 0, 255)  # Red for low traffic
        message = "Signal Turn OFF"

    # Create a filled polygon with transparency
    overlay = frame.copy()
    cv2.fillPoly(overlay, [polygon_points], color)
    
    # Blend the overlay with the original frame
    alpha = 0.4  # Transparency factor
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    # Draw polygon outline
    cv2.polylines(frame, [polygon_points], isClosed=True, color=(255, 255, 255), thickness=2)

    # Display the signal status in an attractive manner
    cv2.putText(frame, message, (frame_width // 2 - 150, frame_height // 5),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 5, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()


WARNING  imgsz=[1704] must be multiple of max stride 32, updating to [1728]
0: 992x1728 5 bicycles, 7 cars, 6 buss, 1 truck, 863.7ms
Speed: 23.9ms preprocess, 863.7ms inference, 2.0ms postprocess per image at shape (1, 3, 992, 1728)

WARNING  imgsz=[1704] must be multiple of max stride 32, updating to [1728]
0: 992x1728 4 bicycles, 6 cars, 6 buss, 1 truck, 870.7ms
Speed: 19.9ms preprocess, 870.7ms inference, 2.0ms postprocess per image at shape (1, 3, 992, 1728)

WARNING  imgsz=[1704] must be multiple of max stride 32, updating to [1728]
0: 992x1728 2 bicycles, 5 cars, 5 buss, 1 truck, 855.7ms
Speed: 19.0ms preprocess, 855.7ms inference, 3.0ms postprocess per image at shape (1, 3, 992, 1728)

WARNING  imgsz=[1704] must be multiple of max stride 32, updating to [1728]
0: 992x1728 1 bicycle, 8 cars, 1 motorcycle, 5 buss, 2 trucks, 942.0ms
Speed: 16.0ms preprocess, 942.0ms inference, 3.0ms postprocess per image at shape (1, 3, 992, 1728)

WARNING  imgsz=[1704] must be multiple of max str

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
import time

# Load a YOLO model
model = YOLO("yolo11s.pt")

# Open the input video
input_video_path = "6.mp4"
output_video_path = "output_video6.mp4"
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Keep the original FPS but update every 60 frames
frame_counter = 0
update_interval = 2

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Define a threshold for traffic density
traffic_density_threshold = 8  # Adjust this value based on your needs

# Initialize variables for polygon drawing
polygon_points = []
drawing_polygon = False
vehicle_count = 0  # Store vehicle count for every update interval

# Define durations for signal ON and OFF states
signal_off_duration = 11  # Duration in seconds for signal to stay OFF
signal_on_duration =  5 # Duration in seconds for signal to stay ON (example value, adjust as needed)
total_cycle_duration = signal_off_duration + signal_on_duration  # Total cycle duration

# Variables to track the start time of the signal
cycle_start_time = None
signal_state = "OFF"

# Mouse callback function to draw polygon
def draw_polygon(event, x, y, flags, param):
    global drawing_polygon, polygon_points, frame

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing_polygon = True
        polygon_points.append((x, y))
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

    elif event == cv2.EVENT_MOUSEMOVE and drawing_polygon:
        if len(polygon_points) > 1:
            temp_frame = frame.copy()
            cv2.line(temp_frame, polygon_points[-1], (x, y), (0, 255, 0), 2)
            cv2.imshow('Draw Polygon', temp_frame)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing_polygon = False
        polygon_points.append((x, y))
        if len(polygon_points) > 1:
            cv2.line(frame, polygon_points[-2], polygon_points[-1], (0, 255, 0), 2)

# Read the first frame to draw the polygon
ret, frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    exit()

# Create a window and bind the mouse callback function
cv2.namedWindow('Draw Polygon')
cv2.setMouseCallback('Draw Polygon', draw_polygon)

# Draw the polygon on the first frame
while True:
    cv2.imshow('Draw Polygon', frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # Press 'ESC' to exit
        break
    elif k == 13:  # Press 'Enter' to confirm the polygon
        break

cv2.destroyAllWindows()

# Check if the polygon was drawn
if len(polygon_points) < 3:
    print("Error: Polygon must have at least 3 points.")
    cap.release()
    out.release()
    exit()

# Convert the list of points to a NumPy array and ensure they are integers
polygon_points = np.array(polygon_points, dtype=np.int32)
polygon_points = polygon_points.reshape((-1, 1, 2))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    
    # Only update vehicle count every 60 frames
    if frame_counter % update_interval == 0:
        # Convert frame to RGB for YOLO
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform prediction
        results = model.predict(frame_rgb, imgsz=640, classes=[1, 2, 3, 5, 7])

        vehicle_count = 0  # Reset vehicle count for the update interval

        # Count vehicles inside the polygon
        if isinstance(results, list):
            for result in results:
                for det in result.boxes:
                    x1, y1, x2, y2 = map(int, det.xyxy[0].tolist())
                    
                    # Check if the center of the detected object is inside the polygon
                    box_center = (int((x1 + x2) // 2), int((y1 + y2) // 2))
                    if cv2.pointPolygonTest(polygon_points, box_center, False) >= 0:
                        vehicle_count += 1

    # Check if the signal should be turned ON based on vehicle count
    if vehicle_count > traffic_density_threshold and signal_state == "OFF":
        cycle_start_time = time.time()  # Record the start time of the cycle
        signal_state = "ON"

    # Check if the cycle duration has passed
    if cycle_start_time is not None:
        elapsed_time = time.time() - cycle_start_time
        if elapsed_time >= total_cycle_duration:
            cycle_start_time = time.time()  # Reset the cycle start time
            signal_state = "ON"  # Reset the signal state to ON

    # Determine the signal state based on elapsed time within the cycle
    if cycle_start_time is not None:
        elapsed_time = time.time() - cycle_start_time
        if elapsed_time < signal_off_duration:
            signal_state = "OFF"
        else:
            signal_state = "ON"

    # Determine the color and message based on the signal state
    if signal_state == "ON":
        color = (0, 255, 0)  # Green for high traffic
        message = "Signal Turn ON"
    else:
        color = (0, 0, 255)  # Red for low traffic
        message = "Signal Turn OFF"

    # Create a filled polygon with transparency
    overlay = frame.copy()
    cv2.fillPoly(overlay, [polygon_points], color)
    
    # Blend the overlay with the original frame
    alpha = 0.4  # Transparency factor
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    # Draw polygon outline
    cv2.polylines(frame, [polygon_points], isClosed=True, color=(255, 255, 255), thickness=2)

    # Display the signal status in an attractive manner
    cv2.putText(frame, message, (frame_width // 2 - 150, frame_height // 5),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 5, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 4 bicycles, 8 cars, 3 buss, 349.1ms
Speed: 11.0ms preprocess, 349.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 bicycles, 8 cars, 4 buss, 1 truck, 289.2ms
Speed: 10.0ms preprocess, 289.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 bicycles, 7 cars, 3 buss, 1 truck, 270.3ms
Speed: 3.0ms preprocess, 270.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 bicycles, 7 cars, 3 buss, 1 truck, 260.3ms
Speed: 5.0ms preprocess, 260.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 bicycles, 9 cars, 1 motorcycle, 2 buss, 1 truck, 226.4ms
Speed: 8.0ms preprocess, 226.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 bicycles, 9 cars, 1 motorcycle, 2 buss, 1 truck, 272.3ms
Speed: 4.0ms preprocess, 272.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 bicycles, 9 cars, 2 motorcycles, 3 